In [153]:
import numpy as np
import pandas as pd
from torch_geometric.data import Data

# PyTorch Geometric custom Dataset
This module illustrates the process part of the custom dataset creation for PyG done in the `datasets/SP100Stocks.py` file.
The Dataset class is used to create a custom dataset and use it with PyG modules.
**Note: To use this notebook, first execute notebooks 1 and 2.**

## Processing the graph data
The next cells demonstrate the adjacency matrix and historical values processing done in the `Dataset.process(*)` method.   

### Opening the historical values and the adjacency matrix
The goal is here to create T (the number of timestamps) graphs, one for each timestep. We use the same adjacency matrix for all the graphs as the structure is based on fundamentals and will not change.

In [154]:
values = pd.read_csv('../data/SP100/raw/values.csv').set_index(['Symbol', 'Date'])
values.head()

Close    Volume  Variation       RSI  \
Symbol Date                                                                  
AAPL   2019-06-11 00:00:00-04:00  47.066399  0.195742  -0.000257  0.594733   
       2019-06-12 00:00:00-04:00  46.916603 -0.472418   0.001237  0.583426   
       2019-06-13 00:00:00-04:00  46.906933 -0.209040  -0.002825  0.582656   
       2019-06-14 00:00:00-04:00  46.566288 -0.433289   0.006212  0.554871   
       2019-06-17 00:00:00-04:00  46.844131 -0.748321   0.005132  0.572767   

                                      MACD  BhB  BlB  
Symbol Date                                           
AAPL   2019-06-11 00:00:00-04:00 -0.578354  0.0  0.0  
       2019-06-12 00:00:00-04:00 -0.379584  0.0  0.0  
       2019-06-13 00:00:00-04:00 -0.220298  0.0  0.0  
       2019-06-14 00:00:00-04:00 -0.120164  0.0  0.0  
       2019-06-17 00:00:00-04:00 -0.018179  0.0  0.0

In [155]:
adj = np.load('../data/SP100/raw/adj.npy')
adj[:5, :5]

array([[0.        , 0.        , 0.        , 0.25805462, 0.25805462],
       [0.        , 0.        , 0.51610923, 0.        , 0.        ],
       [0.        , 0.51610923, 0.        , 0.        , 0.        ],
       [0.25805462, 0.        , 0.        , 0.        , 0.51610923],
       [0.25805462, 0.        , 0.        , 0.51610923, 0.        ]])

### Creating the feature matrix and edge indices
The feature matrix is a 3d matrix of shape (stocks_nb, timestamps_nb, features_nb). The adjacency matrix is used to create the edge_index matrix in the PyTorch Geomtric format.

In [156]:
nodes_nb = len(adj)
x = np.array(
	values.drop(columns=["Close"]).to_numpy().reshape((nodes_nb, -1, values.shape[1] - 1))
)
x = np.swapaxes(x, 1, 2)  # to obtain (nodes, features, timestamps)

edge_nb = np.count_nonzero(adj) // 2  # the graph is undirected
edge_index = np.zeros((2, edge_nb))
edge_weight = np.zeros((edge_nb,))
count = 0
for i in range(nodes_nb):
	for j in range(i + 1, nodes_nb):
		if (weight := adj[i, j]) != 0:
			edge_index[0, count], edge_index[1, count] = i, j
			edge_weight[count] = weight
			count += 1
x.shape, edge_index.shape, edge_weight.shape

((100, 6, 1234), (2, 1230), (1230,))

### Building the graphs
Finally, the graphs can be built using the Data class for PyG.

In [157]:
timestamps = [
	Data(x=x[:, :, idx], edge_index=edge_index, edge_weight=edge_weight) for idx in range(x.shape[1])
]
timestamps[:5]

[Data(x=[100, 6], edge_index=[2, 1230], edge_weight=[1230]),
 Data(x=[100, 6], edge_index=[2, 1230], edge_weight=[1230]),
 Data(x=[100, 6], edge_index=[2, 1230], edge_weight=[1230]),
 Data(x=[100, 6], edge_index=[2, 1230], edge_weight=[1230]),
 Data(x=[100, 6], edge_index=[2, 1230], edge_weight=[1230])]

## Defining the PyTorch Geometric datasets
The previously defined method is used to process the data and create a standard PyG Dataset class.
Two datasets are defined:
- The first dataset contains graphs with data for a single timestamps.
- The second dataset contains graphs with 3d node features -- data for the timestamps $[\![t, t+T[\![$ -- and a target variable -- the variation at time $t+T$ -- used for forecasting.

These datasets are used in the next notebooks to train and evaluate the forecasting model. The code is in the `datasets/SP100Stocks.py` and `datasets/SP100StocksForecasting.py` file.